In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

In [ ]:
cover.open_cover(device_id="102d5c721979848782b13638a7aaa932")

In [3]:
cover.close_cover(device_id="102d5c721979848782b13638a7aaa932")

In [2]:
cover.bathroom_window_opener.open_cover()

In [7]:
cover.bathroom_window_opener

'open'

In [7]:
HUMIDITY_LOWER_THRESHOLD = 55
HUMIDITY_UPPER_THRESHOLD = 60
HUMIDITY_HIGH_HOLD_SECONDS = 0
HUMIDITY_LOW_HOLD_SECONDS = 5*60
MANUALLY_OPEN_SECONDS = 15*60
MANUALLY_CLOSED_SECONDS = 10*60

from enum import Enum
class OperationMode:
    MANUAL = 1
    AUTO = 2
    AUTO_SUSPENDED = 3


operation_mode = OperationMode.AUTO
auto_mode_open_window = False


@time_trigger
def on_startup():
    input_boolean.bathroom_window_opener_automatically = 'on'

In [8]:
def actuate_window(open_window):
    if open_window:
        cover.bathroom_window_opener.open_cover()
    else:
        cover.bathroom_window_opener.close_cover()
  
   
@state_trigger("float(sensor.lumi_lumi_weather_45e15906_humidity) > float(HUMIDITY_UPPER_THRESHOLD)", state_hold=HUMIDITY_HIGH_HOLD_SECONDS)
def humidity_high_trigger(value=None, old_value=None):
    global operation_mode
    global auto_mode_open_window

#    humidity = float(value)
    log.debug(f"humidity high: {value}")
    
    auto_mode_open_window = True
    if operation_mode == OperationMode.AUTO:
        actuate_window(auto_mode_open_window)
        
        
@state_trigger("float(sensor.lumi_lumi_weather_45e15906_humidity) < float(HUMIDITY_LOWER_THRESHOLD)", state_hold=HUMIDITY_LOW_HOLD_SECONDS)
def humidity_low_trigger(value=None, old_value=None):
    global operation_mode
    global auto_mode_open_window

#    humidity = float(value)
    log.debug(f"humidity low: {value}")
    
    auto_mode_open_window = False
    if operation_mode == OperationMode.AUTO:
        actuate_window(auto_mode_open_window)

humidity high: 70
humidity high: 70
humidity low: 40


In [24]:
import datetime
log.debug(datetime.datetime.now())

2021-06-14 08:50:19.038349


In [20]:
def set_auto_window_state(humidity):  
    if humidity > humidity_upper_threshold:
        log.info(f"Automatically opening window at humidity: {humidity}")
        auto_mode_open_window = True
    elif humidity < humidity_lower_threshold:
        log.info(f"Automatically closing window at humidity: {humidity}")
        auto_mode_open_window = False

                
def actuate_window(open_window):
    if open_window:
        cover.bathroom_window_opener.open_cover()
    else:
        cover.bathroom_window_opener.close_cover()
  
   
@state_trigger("sensor.lumi_lumi_weather_45e15906_humidity")
def humidity_trigger(value=None, old_value=None):
    humidity = float(value)
    log.debug(f"humidity changed: {humidity}")
    
    set_auto_window_state(humidity)
    if operation_mode == OperationMode.AUTO:
        actuate_window(auto_mode_open_window)

In [26]:
@service("bathroom_window_controller.open_window_manually")
def open_window_manually():
    task.unique("window_manually")

    log.info("Open Window Manually")
    if operation_mode == OperationMode.MANUAL:
        actuate_window(True)
    else:
        operation_mode = OperationMode.AUTO_SUSPENDED
        actuate_window(True)
        task.sleep(manually_open_seconds)
        operation_mode = OperationMode.AUTO
        actuate_window(auto_mode_open_window)
    
    
@service("bathroom_window_controller.close_window_manually")
def close_window_manually():
    task.unique("window_manually")

    if operation_mode == OperationMode.MANUAL:
        actuate_window(False)
    else:
        operation_mode = OperationMode.AUTO_SUSPENDED
        actuate_window(False)
        task.sleep(manually_closed_seconds)
        operation_mode = OperationMode.AUTO
        actuate_window(auto_mode_open_window)
    
    
@service("bathroom_window_controller.disable_auto_mode")
def disable_auto_mode():
    task.unique("window_manually")

    log.info("Disable auto mode")
    operation_mode = OperationMode.MANUAL
    
    
@service("bathroom_window_controller.enable_auto_mode")
def enable_auto_mode():
    task.unique("window_manually")

    log.info("Enable auto mode")
    operation_mode = OperationMode.AUTO
    actuate_window(auto_mode_open_window)

2021-06-14 08:53:13.747049
Close Window Manually
Open Window Manually
Close Window Manually
humidity changed: 44.0
Automatically closing window at humidity: 44.0
2021-06-14 09:08:13.750204
